# Convolutional Autoencoder for Non-Pulse-Type GMs
# <font color='blue'>Pre-trained<font color='black'> Model

In [ ]:
# File directory (change the directory to the folder including GMIM_NonPulse_Pretrain.mat)
file_dir = 'C:/Users/jiayi/Desktop/GM Clustering and Selection Codes for GitHub/'


Pulse_Type = "NonPulse"


Model_Type = "Pretrain"


# Stop training when loss reaching
Loss_Stop = 0.005


# Number of epochs
NUM_EPOCH = 10000


# Batch size (default as 5 times of the number of non-pulse-type GMs, user can select other batch size based on experience)
BATCH_SIZE = 180


# Learning rate for Adam optimizer
Learning_Rate = 0.001


# Drop learning rate
Drop_Rate1 = 0.85
NUM_EPOCH_DROP1 = 1000

Drop_Rate2 = 0.90
NUM_EPOCH_DROP2 = 2000

Drop_Rate3 = 0.95
NUM_EPOCH_DROP3 = 3000

Drop_Rate4 = 1.00


# Drop learning rate every Drop_Epoch
Drop_Epoch = 250


# Latent space size
latent_dim = 5

In [ ]:
# Activation function
from keras.layers import ReLU
from keras.layers import PReLU
from keras.layers import LeakyReLU
# Activation function other than latent space 
# act_fun = 'ReLU'
# act_fun = 'PReLU'
act_fun = LeakyReLU(alpha=0.3)
# act_fun = 'tanh'

# Activation function for latent space 
# act_fun_LS = 'ReLU'
# act_fun_LS = 'PReLU'
act_fun_LS = LeakyReLU(alpha=0.3)
# act_fun_LS = 'tanh'


# Kernel size
KERNEL_SIZE1 = 10
KERNEL_SIZE2 = 5


# Max pooling size
POOL_SIZE1 = 5
POOL_SIZE2 = 3


# Load Sa data
import scipy.io
Data_Name = 'GMIM_' + Pulse_Type + '_' + Model_Type + '.mat' 
# print('Input GMs = ' + Data_Name)
GMIM_Raw = scipy.io.loadmat(file_dir + Data_Name)


# Define a GPU usage (default as CPU)
# CPU = -1, GPU0 = 0
GPU_ID = -1
import os
os.environ['CUDA_VISIBLE_DEVICES'] = str(GPU_ID) 


# Folder name
ID = f"_{Pulse_Type}_{Model_Type}" 

In [ ]:
# Check if the GPU is specified
import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

In [ ]:
# Create folders for model, data, and figure
import shutil

model_dir = f"Model{str(ID)}" 
modelExist = os.path.exists(model_dir)
if modelExist:
    shutil.rmtree(model_dir)
    os.makedirs(model_dir) 
    # print('Model folder exists, delete the current files in this folder.')
else:
    os.makedirs(model_dir)    
    # print('Model folder does not exist, create this folder.')
    
data_dir = f"Data{str(ID)}" 
dataExist = os.path.exists(data_dir)
if dataExist:
    shutil.rmtree(data_dir)
    os.makedirs(data_dir) 
    # print('Data folder exists, delete the current files in this folder.')
else:
    os.makedirs(data_dir)    
    # print('Data folder does not exist, create this folder.')
        
figure_dir = f"Figure{str(ID)}" 
figureExist = os.path.exists(figure_dir)
if figureExist:
    shutil.rmtree(figure_dir)    
    os.makedirs(figure_dir) 
    # print('Figure folder exists, delete the current files in this folder.')
else:
    os.makedirs(figure_dir)    
    # print('Figure folder does not exist, create this folder.')

In [ ]:
# Load packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Reshape, Conv1DTranspose, Lambda, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.losses import mse
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
import scipy.io
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import time

In [ ]:
# Load Sa
Sa = GMIM_Raw["LogSa"]     

# Check the dimensions of Sa
print('Sa   shape:', Sa.shape)
print('')

In [ ]:
# Autoencoder Sa
input_Sa = Input(shape=(Sa.shape[1],1), name = 'x_Sa')


# Convolutional and max pooling layers for Sa
x_Sa_C1 = Conv1D(filters = 32, kernel_size = KERNEL_SIZE1, padding="same",
                 activation=act_fun, name = 'x_Sa_C1',
                 kernel_initializer=initializers.HeNormal,
                 bias_initializer=initializers.Zeros())(input_Sa)

x_Sa_P1 = MaxPooling1D(pool_size=POOL_SIZE1, strides=2, padding='same', name = 'x_Sa_P1')(x_Sa_C1)

x_Sa_C2 = Conv1D(filters = 16, kernel_size = KERNEL_SIZE1, padding="same", 
                 activation=act_fun, name = 'x_Sa_C2',
                 kernel_initializer=initializers.HeNormal,
                 bias_initializer=initializers.Zeros())(x_Sa_P1)

x_Sa_P2 = MaxPooling1D(pool_size=POOL_SIZE2, strides=2, padding='same', name = 'x_Sa_P2')(x_Sa_C2)

x_Sa_C3 = Conv1D(filters = 8, kernel_size = KERNEL_SIZE2, padding="same",
                 activation=act_fun, name = 'x_Sa_C3',
                 kernel_initializer=initializers.HeNormal,
                 bias_initializer=initializers.Zeros())(x_Sa_P2)

# Flatten layer for Sa
x_Sa_F = Flatten(name = 'x_Sa_F')(x_Sa_C3)


# Dense layers for Sa
x_Sa_D1 = Dense(256, activation=act_fun, name = 'x_Sa_D1',
                      kernel_initializer=initializers.HeNormal,
                      bias_initializer=initializers.Zeros())(x_Sa_F)
x_Sa_D2 = Dense(128, activation=act_fun, name = 'x_Sa_D2',
                     kernel_initializer=initializers.HeNormal,
                     bias_initializer=initializers.Zeros())(x_Sa_D1)
x_Sa_D3 = Dense(64, activation=act_fun, name = 'x_Sa_D3',
                    kernel_initializer=initializers.HeNormal,
                    bias_initializer=initializers.Zeros())(x_Sa_D2)
x_Sa_D4 = Dense(32, activation=act_fun, name = 'x_Sa_D4',
                    kernel_initializer=initializers.HeNormal,
                    bias_initializer=initializers.Zeros())(x_Sa_D3)
x_Sa_D5 = Dense(16, activation=act_fun, name = 'x_Sa_D5',
                    kernel_initializer=initializers.HeNormal,
                    bias_initializer=initializers.Zeros())(x_Sa_D4)
x_Sa_D6 = Dense(8, activation=act_fun, name = 'x_Sa_D6',
                   kernel_initializer=initializers.HeNormal,
                   bias_initializer=initializers.Zeros())(x_Sa_D5)


# Latent feature layer
z = Dense(latent_dim, activation=act_fun_LS, name = 'Encoder',
                      kernel_initializer=initializers.HeNormal,
                      bias_initializer=initializers.Zeros())(x_Sa_D6)


# Denses layer for Sa
y_Sa_D1 = Dense(8, activation=act_fun, name = 'y_Sa_D1',
                   kernel_initializer=initializers.HeNormal,
                   bias_initializer=initializers.Zeros())(z)
y_Sa_D2 = Dense(16, activation=act_fun, name = 'y_Sa_D2',
                    kernel_initializer=initializers.HeNormal,
                    bias_initializer=initializers.Zeros())(y_Sa_D1)
y_Sa_D3 = Dense(32, activation=act_fun, name = 'y_Sa_D3',
                    kernel_initializer=initializers.HeNormal,
                    bias_initializer=initializers.Zeros())(y_Sa_D2)
y_Sa_D4 = Dense(64, activation=act_fun, name = 'y_Sa_D4',
                    kernel_initializer=initializers.HeNormal,
                    bias_initializer=initializers.Zeros())(y_Sa_D3)
y_Sa_D5 = Dense(128, activation=act_fun, name = 'y_Sa_D5',
                     kernel_initializer=initializers.HeNormal,
                     bias_initializer=initializers.Zeros())(y_Sa_D4)
y_Sa_D6 = Dense(256, activation=act_fun, name = 'y_Sa_D6',
                     kernel_initializer=initializers.HeNormal,
                     bias_initializer=initializers.Zeros())(y_Sa_D5)
y_Sa_D7 = Dense(x_Sa_F.shape[1], activation=act_fun, name = 'y_Sa_D7',
                                 kernel_initializer=initializers.HeNormal,
                                 bias_initializer=initializers.Zeros())(y_Sa_D6)


# Inverse flatten layer
y_Sa_F = Reshape([round(Sa.shape[1]/4), 8], name = 'y_Sa_F')(y_Sa_D7)


# Deconvolutional layers
y_Sa_C1 = Conv1DTranspose(filters = 16, kernel_size = KERNEL_SIZE2, padding="same", 
                          activation=act_fun, name = 'y_Sa_C1',
                          kernel_initializer=initializers.HeNormal,
                          bias_initializer=initializers.Zeros())(y_Sa_F)

y_Sa_C2 = Conv1DTranspose(filters = 16, strides = 2, kernel_size = KERNEL_SIZE2, padding="same", 
                          activation=act_fun, name = 'y_Sa_C2',
                          kernel_initializer=initializers.HeNormal,
                          bias_initializer=initializers.Zeros())(y_Sa_C1)                                                     

y_Sa_C3 = Conv1DTranspose(filters = 32, kernel_size = KERNEL_SIZE1, padding="same", 
                          activation=act_fun, name = 'y_Sa_C3',
                          kernel_initializer=initializers.HeNormal,
                          bias_initializer=initializers.Zeros())(y_Sa_C2)                                                     

y_Sa_C4 = Conv1DTranspose(filters = 32, strides = 2, kernel_size = KERNEL_SIZE1, padding="same", 
                          activation=act_fun, name = 'y_Sa_C4',
                          kernel_initializer=initializers.HeNormal,
                          bias_initializer=initializers.Zeros())(y_Sa_C3)   

y_Sa_C5 = Conv1DTranspose(filters = 1, kernel_size = KERNEL_SIZE1, padding="same", 
                          activation=act_fun, name = 'y_Sa_C5',
                          kernel_initializer=initializers.HeNormal,
                          bias_initializer=initializers.Zeros())(y_Sa_C4)    

output_Sa = Conv1DTranspose(filters = 1, kernel_size = KERNEL_SIZE1, padding="same", 
                            activation='linear', name = 'y_Sa',
                            kernel_initializer=initializers.HeNormal,
                            bias_initializer=initializers.Zeros())(y_Sa_C5) 


autoencoder = Model([input_Sa], [output_Sa])

In [ ]:
autoencoder.summary()

In [ ]:
plot_model(autoencoder, figure_dir + "/Autoencoder.png", show_shapes=True)

In [ ]:
# Custom loss function
loss1 = mse(input_Sa, output_Sa)
autoencoder.add_loss(loss1) 

In [ ]:
# Custom optimizer leanring rate
autoencoder.compile(optimizer=optimizers.legacy.Adam(learning_rate=Learning_Rate))

In [ ]:
# Leanring rate step decay
def lr_step_decay(epoch, lr):
    if epoch < NUM_EPOCH_DROP1:
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(epoch/Drop_Epoch))
    if ((epoch >= NUM_EPOCH_DROP1) and (epoch < NUM_EPOCH_DROP2)):
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(NUM_EPOCH_DROP1/Drop_Epoch)) * math.pow(Drop_Rate2, math.floor((epoch-NUM_EPOCH_DROP1)/Drop_Epoch))
    if ((epoch >= NUM_EPOCH_DROP2) and (epoch < NUM_EPOCH_DROP3)):
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(NUM_EPOCH_DROP1/Drop_Epoch)) * math.pow(Drop_Rate2, math.floor((NUM_EPOCH_DROP2-NUM_EPOCH_DROP1)/Drop_Epoch)) * math.pow(Drop_Rate3, math.floor((epoch-NUM_EPOCH_DROP2)/Drop_Epoch))
    if epoch >= NUM_EPOCH_DROP3:
        return Learning_Rate * math.pow(Drop_Rate1, math.floor(NUM_EPOCH_DROP1/Drop_Epoch)) * math.pow(Drop_Rate2, math.floor((NUM_EPOCH_DROP2-NUM_EPOCH_DROP1)/Drop_Epoch)) * math.pow(Drop_Rate3, math.floor((NUM_EPOCH_DROP3-NUM_EPOCH_DROP2)/Drop_Epoch)) * math.pow(Drop_Rate4, math.floor((epoch-NUM_EPOCH_DROP3)/Drop_Epoch))

              
LR_Scheduler = tf.keras.callbacks.LearningRateScheduler(lr_step_decay, verbose=0)

In [ ]:
# Checkpoint path and directory
checkpoint_path = model_dir + "/Checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
# Callback to save the model's weights at the best epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 verbose = 1, 
                                                 monitor='loss',
                                                 mode='min',
                                                 save_weights_only = True,
                                                 save_best_only = True)

In [ ]:
# Stop training when loss reaches the user-defined value
class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') <= Loss_Stop):
            print("\n\n\nReached %s loss value, so stopping training!\n\n\n"  %(str(Loss_Stop)))
            self.model.stop_training = True

trainingStopCallback = haltCallback()

In [ ]:
# Train the model
Input_set = [Sa.reshape(Sa.shape[0], Sa.shape[1],1)]
Output_set = [Sa.reshape(Sa.shape[0], Sa.shape[1],1)]

start = time.time()
history = autoencoder.fit(Input_set, Output_set,
                          epochs = NUM_EPOCH,
                          batch_size = BATCH_SIZE,
                          callbacks = [cp_callback, trainingStopCallback, LR_Scheduler],
                          verbose = 0)

        
end = time.time()

In [ ]:
# Running time
print("The running time:", round((end-start)/60,1), "mins")
print('')

Time = np.asarray(round((end-start)/60,1))
Time = Time.reshape(1,-1)
np.savetxt(data_dir + "/Running_Time.txt", Time);

# Find the best number of epochs to avoid overfitting
val_loss_min_index = history.history['loss']. index(min(history.history['loss']))
ckpt_best = val_loss_min_index+1
print('Number of epochs at the minimum loss = ', ckpt_best)
print('')

Min_Test_Loss_Epoch = open(data_dir + '/Min_Loss_Epoch.dat', 'w')
Min_Test_Loss_Epoch.write(str(ckpt_best))
Min_Test_Loss_Epoch.close()

# Save loss
np.savetxt(data_dir + '/Loss.dat', history.history['loss'])

# Plot loss
fig, ax = plt.subplots()

plt.plot(np.arange(1,len(history.history['loss'])+1), history.history['loss'],'k-',linewidth=2,label='Train loss')
plt.legend(fontsize=20)

plt.xlim([0, len(history.history['loss'])+1])

ax.set_xlabel('Epoch', fontsize=20)
ax.set_ylabel('Loss', fontsize=20) 
ax.set_yscale('log')

ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)

fig.set_size_inches(10, 7)

plt.savefig(figure_dir + '/Loss.jpg', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
# Best checkpoint ID
Checkpoint_ID = os.path.join(model_dir, "Checkpoint.ckpt")

# Save decoded data
autoencoder_ckpt_best = autoencoder;
autoencoder_ckpt_best.load_weights(Checkpoint_ID);
All_decoded = autoencoder_ckpt_best.predict([Sa.reshape(Sa.shape[0], Sa.shape[1],1)],verbose = 0);
Sa_decoded = All_decoded.reshape(All_decoded.shape[0], All_decoded.shape[1]);
np.savetxt(data_dir + '/Decode_Sa.dat', Sa_decoded);

# Save encoded data
layer_name_Encoder = 'Encoder'
encoder_ckpt_best_Encoder = Model(inputs = autoencoder_ckpt_best.input, outputs = autoencoder_ckpt_best.get_layer(layer_name_Encoder).output);
All_encoded_Encoder = encoder_ckpt_best_Encoder.predict([Sa.reshape(Sa.shape[0], Sa.shape[1],1)],verbose = 0);
np.savetxt(data_dir + '/Encode.dat', All_encoded_Encoder);


In [ ]:
# Plot Sa
fig, ax = plt.subplots()
In_plot = np.exp(np.array(Sa).flatten());
Out_plot = np.exp(np.array(Sa_decoded).flatten());
Rho = np.corrcoef(In_plot,Out_plot) 
MSE = np.square(np.subtract(In_plot,Out_plot)).mean()

plt.xlim([min(min(In_plot),min(Out_plot))-0.05, max(max(In_plot),max(Out_plot))+0.05])
plt.ylim([min(min(In_plot),min(Out_plot))-0.05, max(max(In_plot),max(Out_plot))+0.05])
ax.plot([min(min(In_plot),min(Out_plot))-0.05, max(max(In_plot),max(Out_plot))+0.05], [min(min(In_plot),min(Out_plot))-0.05, max(max(In_plot),max(Out_plot))+0.05], 
        ls="--", color = ".3")

ax.plot([0, 0], [min(min(In_plot),min(Out_plot))-0.05, max(max(In_plot),max(Out_plot))+0.05], ls="-.", color =".3")
ax.plot([min(min(In_plot),min(Out_plot))-0.05, max(max(In_plot),max(Out_plot))+0.05], [0, 0], ls="-.", color =".3")


plt.scatter(In_plot, Out_plot, marker='o', color="b", alpha=0.3, label= r'$\rho$ = %.4f,  MSE = %.4f' %(Rho[0,1], MSE))
plt.legend(fontsize=20,loc='upper left')

ax.set_xlabel('Input', fontsize=20)
ax.set_ylabel('Output', fontsize=20) 

ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
ax.set_title('Sa', fontsize=20)
fig.set_size_inches(10, 10)

plt.savefig(figure_dir + '/Q-Q Sa.jpg', dpi=600, bbox_inches='tight')
plt.show()